In [1]:
import pandas as pd
import json
import re
from collections import defaultdict

In [2]:
excel_path = '../data/test_data_v2.xlsx'
xlsx = pd.ExcelFile(excel_path)

for sheet_name in xlsx.sheet_names:
    print(sheet_name)

3
6
9
12


In [3]:
# Define a function to parse each sheet
def parse_sheet(df):
    result = defaultdict(lambda: defaultdict(dict))

    # Identify all columns related to expression values
    expr_cols = [col for col in df.columns if 'log2FC_cluster' in col]

    # Loop through each row in the sheet
    for _, row in df.iterrows():
        organelle = row["Organelle"]
        genotype = row["Genotype"]
        num_clusters = row["Significantly expressed in NO. of clusters"]

        # Build Gene Info dictionary
        gene_info = {
            "Name": row["GeneName"],
            "Description": row["Description"],
            "TF Family": row["TF Family"]  # You can change/remove this if needed
        }

        # Create the main gene entry
        gene_entry = {
            "Gene Info": gene_info,
            "Number of cluster Significantly expressed": num_clusters,
            "Cell type": defaultdict(dict)
        }

        # Loop through expression columns to extract values
        for col in expr_cols:
            parts = col.split("|")
            if len(parts) < 2:
                continue

            cell_type = parts[0].strip()
            cluster_match = re.search(r"log2FC_cluster\s*(\d+)", col)
            if cluster_match:
                cluster = cluster_match.group(1)
                expression_value = row[col]
                gene_entry["Cell type"][cell_type][f"Cluster {cluster}"] = expression_value

        # Nest entries into organelle → genotype
        result[organelle][genotype][row["GeneID"]] = gene_entry

    return result

In [4]:
# Build the final JSON from all sheets
final_json = {}
for sheet_name in xlsx.sheet_names:
    df = xlsx.parse(sheet_name)
    final_json[sheet_name] = parse_sheet(df)

In [5]:
# Export to JSON file
output_path = "../data/test_data.json"
with open(output_path, "w") as f:
    json.dump(final_json, f, indent=2)